In [2]:
# VENDO COM A CÂMERA ABERTA

import cv2
import supervision as sv
from ultralytics import YOLO
import os

path = 'C:\\Users\\ct67ca\\Desktop\\Cana\\runs\\train4\\weights\\best.pt'
model = YOLO(path)

cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Erro ao abrir a câmera.")
    exit()

# Inicializa os anotadores
box_annotator = sv.BoxAnnotator()
label_annotator = sv.LabelAnnotator()

while True:
    # Captura o quadro da câmera
    ret, frame = cap.read()

    if not ret:
        print("Erro ao capturar o quadro.")
        break

    # Executa a detecção de objetos no quadro
    results = model(frame)[0]
    detections = sv.Detections.from_ultralytics(results)

    # Cria os rótulos
    labels = [
        f"{class_name} {confidence:.2f}"
        for class_name, confidence
        in zip(detections['class_name'], detections.confidence)
    ]

    # Anota o quadro
    annotated_frame = box_annotator.annotate(scene=frame, detections=detections)
    annotated_frame = label_annotator.annotate(scene=annotated_frame, detections=detections, labels=labels)

    # Exibe o quadro anotado
    cv2.imshow('Câmera - Detecções', annotated_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

def process_image(image_path: str):
    frame = cv2.imread(image_path)

    if frame is None:
        raise ValueError("Erro ao carregar a imagem")
    
    image = model(frame)[0]
    detections = sv.Detections.from_ultralytics(image)

    box_annotator = sv.BoxAnnotator()
    label_annotator = sv.LabelAnnotator()

    labels = [
        f"{class_name} {confidence:.2f}"
        for class_name, confidence
        in zip(detections['class_name'], detections.confidence)
    ]

    annotated_frame = box_annotator.annotate(scene=frame, detections=detections)
    annotated_frame = label_annotator.annotate(scene=annotated_frame, detections=detections, labels=labels)

    processed_image_path = f"processed/processed_{os.path.basename(image_path)}"

    cv2.imwrite(processed_image_path, annotated_frame)

    return processed_image_path



0: 608x800 (no detections), 54.8ms
Speed: 3.0ms preprocess, 54.8ms inference, 0.8ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 63.1ms
Speed: 5.0ms preprocess, 63.1ms inference, 0.0ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 60.8ms
Speed: 2.0ms preprocess, 60.8ms inference, 0.0ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 58.3ms
Speed: 3.0ms preprocess, 58.3ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 56.2ms
Speed: 4.0ms preprocess, 56.2ms inference, 0.0ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 54.0ms
Speed: 3.0ms preprocess, 54.0ms inference, 0.0ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 53.1ms
Speed: 3.0ms preprocess, 53.1ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 800)

0: 608x800 (no detections), 52.3ms
Speed: 4.0ms preprocess, 52.3ms i

In [6]:
# VENDO COM IMAGENS IMPORTADAS

import cv2
import supervision as sv
from ultralytics import YOLO
from flask import Flask, request, render_template, send_from_directory, Response
import threading
import os

app = Flask(__name__)

# pasta para onde vão os uploads
UPLOAD_FOLDER = 'uploads'

# pasta para onde vão os resultados
RESULTS_FOLDER = 'results'
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
os.makedirs(RESULTS_FOLDER, exist_ok=True)

cap = None 
frame = None

# caminho do analisador
path = 'C:\\Users\\ct67ca\\Desktop\\Cana\\runs\\train4\\weights\\best.pt'
model = YOLO(path) # modelo YOLO

# rota para abrir a página html
# @app.route('/')
# def upload():
#     return render_template('pagina.html') # pagina principal

# rota de upload das imagens
@app.route('/upload', methods=['POST'])
def upload_image():
    # erro se o arquivo não for encontrado no request
    if 'file' not in request.files:
        return 'Nenhum arquivo encontrado', 400
    
    # erro se nenhum arquivo for selecionado (sem nome)
    file = request.files['file']
    if file.filename == '':
        return 'Nenhum arquivo selecionado', 400

    # Se não cair em nenhuma condicional, a imagem é salva e processada
    image_path = os.path.join(UPLOAD_FOLDER, file.filename)
    file.save(image_path)
    
    # erro se não conseguir abrir a imagem (frame)
    frame = cv2.imread(image_path)
    if frame is None:
        return 'Erro ao abrir a imagem', 400
    
    # função que anota os quadros
    box_annotator = sv.BoxAnnotator()

    # função que anota os labels
    label_annotator = sv.LabelAnnotator()

    image = model(frame)[0]

    # de fato realizando as detecções das imagens com o modelo YOLO
    detections = sv.Detections.from_ultralytics(image)

    # labels que aparecem no print
    labels = [
        f"{class_name}{confidence: .2f}"
        for class_name, confidence in zip(detections['class_name'], detections.confidence)
    ]

    # de fato anotando de acordo com as detecções
    annotated_frame = box_annotator.annotate(scene=frame, detections=detections)
    annotated_frame = label_annotator.annotate(scene=annotated_frame, detections=detections, labels=labels)

    # salvando a imagem processada
    result_path = os.path.join(RESULTS_FOLDER, file.filename)
    cv2.imwrite(result_path, annotated_frame)

    return render_template('pagina_de_resultados.html', filename=file.filename)

@app.route('/results/<filename>')
def get_result(filename):
    return send_from_directory(RESULTS_FOLDER, filename)

@app.route('/camera')
def open_camera():
    global cap
    cap = cv2.VideoCapture(0)
    return Response(generate_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')

def generate_frames():
    global cap
    if not cap.isOpened():
        print("Erro ao abrir a câmera")
        return

     # função que anota os quadros
    box_annotator = sv.BoxAnnotator()

    # função que anota os labels
    label_annotator = sv.LabelAnnotator()

    while True:
        success, frame = cap.read()
        if not success:
            break

        # detecção de fato com a câmera aberta
        results = model(frame)[0]
        detections = sv.Detections.from_ultralytics(results)

        labels = [
            f"{class_name}{confidence:.2f}"
            for class_name, confidence in zip(detections['class_name'], detections.confidence)
        ]

        annotated_frame = box_annotator.annotate(scene=frame, detections=detections)
        annotated_frame = label_annotator.annotate(scene=annotated_frame, detections=detections, labels=labels)

        ret, buffer = cv2.imencode('.jpg', annotated_frame)
        frame = buffer.tobytes()

        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')

@app.route('/stop_camera')
def stop_camera():
    global cap
    if cap is not None:
        cap.release()
    return "Câmera parada."

if __name__ == '__main__':
    app.run(debug=True)


# frame = cv2.imread(image_path)

# if frame is None:
#     print("Erro ao abrir a imagem.")
#     exit()

# # Inicializa os anotadores
# box_annotator = sv.BoxAnnotator()
# label_annotator = sv.LabelAnnotator()

# image = model(frame)[0]
# detections = sv.Detections.from_ultralytics(image)

# labels = [
#         f"{class_name} {confidence:.2f}"
#         for class_name, confidence
#         in zip(detections['class_name'], detections.confidence)
#     ]

# annotated_frame = box_annotator.annotate(scene=frame, detections=detections)
# annotated_frame = label_annotator.annotate(scene=annotated_frame, detections=detections, labels=labels)

# cv2.imshow('Detecções com imagem', annotated_frame)
# cv2.waitKey(0)
# cv2.destroyAllWindows()



0: 800x800 2 unhealthybuds, 81.2ms
Speed: 6.2ms preprocess, 81.2ms inference, 0.0ms postprocess per image at shape (1, 3, 800, 800)
